In [1]:
import pandas as pd
import numpy as np
from os import listdir
from os.path import isfile, join
import matplotlib.pyplot as plt
import os
from skimage.color.adapt_rgb import adapt_rgb, each_channel, hsv_value
from skimage import filters
from skimage.exposure import rescale_intensity
from skimage.data import camera
from skimage.util import compare_images
from skimage import exposure
from skimage import data, img_as_float
from skimage import exposure

In [2]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn import preprocessing
from sklearn.metrics import classification_report

In [3]:
mypath = '/Users/yinzi/Downloads/crop_part1'
filenames = np.array([f for f in listdir(mypath) if isfile(join(mypath, f))])
splitcolumns = [x.split('_')[0:3] + [mypath +'/' + x] for x in filenames if x.count('_') == 3]
filecolumns = ['age','gender','race','file']
df = pd.DataFrame(data = splitcolumns, columns = filecolumns)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/yinzi/Downloads/crop_part1'

In [4]:
df.head(3)

,age,gender,race,file
0,9,1,2,/Users/yinzi/Downloads/crop_part1/9_1_2_201612...
1,1,1,2,/Users/yinzi/Downloads/crop_part1/1_1_2_201612...
2,16,0,0,/Users/yinzi/Downloads/crop_part1/16_0_0_20170...


In [5]:
df['age']= df['age'].astype('int')
df['gender']= df['gender'].astype('int')
df['race']= df['race'].astype('int')

In [6]:
# put the age in different interval
df['age']=np.where(((df.age>=1) & (df.age<=2)), 0, df.age)
df['age']=np.where(((df.age>=3) & (df.age<6)), 1, df.age)
df['age']=np.where(((df.age>=6) & (df.age<9)), 2, df.age)
df['age']=np.where(((df.age>=9) & (df.age<12)), 3, df.age)
df['age']=np.where(((df.age>=12) & (df.age<21)), 4, df.age)
df['age']=np.where(((df.age>=21) & (df.age<36)), 5, df.age)
df['age']=np.where(((df.age>=36) & (df.age<51)), 6, df.age)
df['age']=np.where(((df.age>=51) & (df.age<80)), 7, df.age)
df['age']=np.where((df.age>=80), 8, df.age)

In [7]:
df['age'].unique()

array([4, 1, 5, 6, 8, 7, 9, 2, 3])

In [8]:
y1 = df['gender']
y2 = df['race']
y3 = df['age']

In [9]:
images = []
for img_path in df['file']:
    image = plt.imread(img_path)
    images.append(image)

In [10]:
imagesArray = np.array(images)

In [11]:
gbc=GradientBoostingClassifier(n_estimators=500,learning_rate=0.05,random_state=100,max_features=5, max_depth = 3)

In [12]:
def sobel_each(image):
    return filters.sobel(image)

In [13]:
def sobel_hsv(image):
    return filters.sobel(image)

In [14]:
# mask 1 & mask 2
def sobelFilterTypes(imagesArray):
    """
    This function takes as input an array of pictures and returns two arrays. The first array has the 
    Sobel filter apply to each picture channel and the second one is applied on the image's hsv. The images are in color
    """
    #filter apply to pictures channel
    filterChannel = []
    #filter apply to hue, saturation, value
    filterHSV = []
    for image in imagesArray:
        image1 = rescale_intensity(1 - sobel_each(image))
        image2 = rescale_intensity(1 - sobel_hsv(image))
        filterChannel.append(image1)
        filterHSV.append(image2)
    return np.array(filterChannel), np.array(filterHSV)

In [15]:
# mask 3 & mask 4
def robertsSobelEdges(imagesArray):
    from skimage import color
    """
    returns two numpy arrays with pictures in black and white and with the edges define using Roberts filter
    on the first one and the Sobel filter on the second one. The images are in black and white
    """
    roberts_ = []
    sobel_ = []
    for image in imagesArray:
        grayImg = color.rgb2gray(image)
        edge_roberts = filters.roberts(grayImg)
        edge_sobel = filters.sobel(grayImg)
        roberts_.append(edge_roberts)
        sobel_.append(edge_sobel)
    return np.array(roberts_), np.array(sobel_)

In [16]:
# mask 5
def exposure_(imagesArray):
    from skimage import exposure
    """
    This fucntion increases the exposure of each picture and it returns a numpy
    array of the modified images. The images are in color
    """
    exp = []
    for image in imagesArray:
        image_eq = exposure.equalize_hist(image)
        exp.append(image_eq)
    return np.array(exp)

In [17]:
# mask 6
def gammaLogaritmictCorrection(imagesArray):
    """
    histogram equalizer 
    """
    gamm = []
    log = []
    for image in imagesArray:
        gamma_corrected = exposure.adjust_gamma(image, 2)
        logarithmic_corrected = exposure.adjust_log(image, 1)
        gamm.append(gamma_corrected)
        log.append(logarithmic_corrected)
    return np.array(gamm), np.array(log)

In [18]:
# mask 7
def histogramEqualizer(imagesArray):
    """
    returns 3 numpy arrays one is with rescale, equalize histogram, adapthis
    
    """
    
    
    
    intensity = []
    hist = []
    adapthist = []
    for image in imagesArray:
        p2, p98 = np.percentile(image, (2, 98))
        
        img_rescale = exposure.rescale_intensity(image, in_range=(p2, p98))

        # Equalization
        img_eq = exposure.equalize_hist(image)

        # Adaptive Equalization
        img_adapteq = exposure.equalize_adapthist(image, clip_limit=0.03)
        
        intensity.append(img_rescale)
        hist.append(img_eq)
        adapthist.append(img_adapteq)
        
    return np.array(intensity), np.array(hist), np.array(adapthist)

### 1.1 Mask:  sobel Filter Channel
### 1.1.1 Mask Sobel1 --- Label: gender

In [19]:
Sobel = list(sobelFilterTypes(imagesArray))
Sobel1 = Sobel[0]
Sobel2 = Sobel[1]

In [21]:
Sobel1.shape

(9778, 200, 200, 3)

In [20]:
nsamples, nx, ny, n = Sobel1.shape
Sobel1_2d = Sobel1.reshape((nsamples,nx*ny*n))

In [21]:
X_Sobel_train, X_Sobel_test, y_Sobel_train, y_Sobel_test = train_test_split(Sobel1_2d, y1, test_size=0.3, random_state=42)

In [22]:
gbc.fit(X_Sobel_train,y_Sobel_train)

GradientBoostingClassifier(learning_rate=0.05, max_features=5, n_estimators=500,
                           random_state=100)

In [23]:
y_Sobel_pred = gbc.predict(X_Sobel_test)

In [24]:
print(confusion_matrix(y_Sobel_test, y_Sobel_pred))

[[ 830  455]
 [ 313 1336]]


In [25]:
GBC_accuracy_score = accuracy_score(y_Sobel_test, y_Sobel_pred)
print("GBC accuracy is %2.2f" % GBC_accuracy_score)

GBC accuracy is 0.74


In [26]:
print(classification_report(y_Sobel_test, y_Sobel_pred))

              precision    recall  f1-score   support

           0       0.73      0.65      0.68      1285
           1       0.75      0.81      0.78      1649

    accuracy                           0.74      2934
   macro avg       0.74      0.73      0.73      2934
weighted avg       0.74      0.74      0.74      2934



### 1.1.2 Mask Sobel1 --- Label race

In [27]:
X_Sobel_train, X_Sobel_test, y_Sobel_train, y_Sobel_test = train_test_split(Sobel1_2d, y2, test_size=0.3, random_state=42)

In [28]:
gbc.fit(X_Sobel_train,y_Sobel_train)

GradientBoostingClassifier(learning_rate=0.05, max_features=5, n_estimators=500,
                           random_state=100)

In [29]:
y_race_Sobel_pred = gbc.predict(X_Sobel_test)

In [30]:
print(confusion_matrix(y_Sobel_test, y_race_Sobel_pred))

[[1476    4   41   46   11]
 [  72   13    5   25    4]
 [ 283    0  140   27    7]
 [ 203    3   18  197   24]
 [ 231    1   19   50   34]]


In [31]:
GBC_accuracy_score = accuracy_score(y_Sobel_test, y_race_Sobel_pred)
print("GBC accuracy is %2.2f" % GBC_accuracy_score)

GBC accuracy is 0.63


In [32]:
print(classification_report(y_Sobel_test, y_race_Sobel_pred))

              precision    recall  f1-score   support

           0       0.65      0.94      0.77      1578
           1       0.62      0.11      0.19       119
           2       0.63      0.31      0.41       457
           3       0.57      0.44      0.50       445
           4       0.42      0.10      0.16       335

    accuracy                           0.63      2934
   macro avg       0.58      0.38      0.41      2934
weighted avg       0.61      0.63      0.58      2934



### 1.1.3 Mask Sobel 1 --- Label age

In [33]:
X_Sobel_train, X_Sobel_test, y_Sobel_train, y_Sobel_test = train_test_split(Sobel1_2d, y3, test_size=0.3, random_state=42)

In [34]:
gbc.fit(X_Sobel_train,y_Sobel_train)

GradientBoostingClassifier(learning_rate=0.05, max_features=5, n_estimators=500,
                           random_state=100)

In [36]:
y_age_Sobel_pred = gbc.predict(X_Sobel_test)

In [37]:
print(confusion_matrix(y_Sobel_test, y_age_Sobel_pred))

[[425  12   0   0   8  21   3  15   1]
 [108  22   2   0   8  36   4  20   2]
 [ 30  13  11   1  25  45   2  19   0]
 [ 12   6   9   4  36  23   1  21   0]
 [ 14   8   3   5  95 137  11  55   1]
 [ 15   3   5   2  46 391  56  99   1]
 [  6   2   1   0  16  97  64 171   1]
 [ 18   2   2   1  18 102  35 376   2]
 [  2   0   1   0   0  20   2  95   8]]


In [38]:
GBC_accuracy_score = accuracy_score(y_Sobel_test, y_age_Sobel_pred)
print("GBC accuracy is %2.2f" % GBC_accuracy_score)

GBC accuracy is 0.48


In [39]:
print(classification_report(y_Sobel_test, y_age_Sobel_pred))

              precision    recall  f1-score   support

           1       0.67      0.88      0.76       485
           2       0.32      0.11      0.16       202
           3       0.32      0.08      0.12       146
           4       0.31      0.04      0.06       112
           5       0.38      0.29      0.33       329
           6       0.45      0.63      0.52       618
           7       0.36      0.18      0.24       358
           8       0.43      0.68      0.53       556
           9       0.50      0.06      0.11       128

    accuracy                           0.48      2934
   macro avg       0.42      0.33      0.32      2934
weighted avg       0.45      0.48      0.43      2934



### 1.2 mask: Filter HSV
### 1.2.1 mask: Filter HSV -- label gender

In [40]:
nsamples, nx, ny, n = Sobel2.shape
Sobel2_2d = Sobel2.reshape((nsamples,nx*ny*n))

In [34]:
X_Sobel2_train, X_Sobel2_test, y_Sobel2_train, y_Sobel2_test = train_test_split(Sobel2_2d, y1, test_size=0.3, random_state=42)

In [35]:
gbc.fit(X_Sobel2_train,y_Sobel2_train)

GradientBoostingClassifier(learning_rate=0.05, max_features=5, n_estimators=500,
                           random_state=100)

In [36]:
y_Sobel2_pred = gbc.predict(X_Sobel2_test)

In [37]:
print(confusion_matrix(y_Sobel2_test, y_Sobel2_pred))

[[ 830  455]
 [ 313 1336]]


In [38]:
GBC_accuracy_score = accuracy_score(y_Sobel2_test, y_Sobel2_pred)
print("GBC accuracy is %2.2f" % GBC_accuracy_score)

GBC accuracy is 0.74


In [39]:
print(classification_report(y_Sobel2_test, y_Sobel2_pred))

              precision    recall  f1-score   support

           0       0.73      0.65      0.68      1285
           1       0.75      0.81      0.78      1649

    accuracy                           0.74      2934
   macro avg       0.74      0.73      0.73      2934
weighted avg       0.74      0.74      0.74      2934



### 1.2.2 mask: filter hsv -- label race

In [41]:
X_Sobel2_train, X_Sobel2_test, y_Sobel2_train, y_Sobel2_test = train_test_split(Sobel2_2d, y2, test_size=0.3, random_state=42)

In [42]:
gbc.fit(X_Sobel2_train,y_Sobel2_train)

GradientBoostingClassifier(learning_rate=0.05, max_features=5, n_estimators=500,
                           random_state=100)

In [43]:
y_race_Sobel2_pred = gbc.predict(X_Sobel2_test)

In [44]:
print(confusion_matrix(y_Sobel2_test, y_race_Sobel2_pred))

[[1476    4   41   46   11]
 [  72   13    5   25    4]
 [ 283    0  140   27    7]
 [ 203    3   18  197   24]
 [ 231    1   19   50   34]]


In [45]:
GBC_accuracy_score = accuracy_score(y_Sobel2_test, y_race_Sobel2_pred)
print("GBC accuracy is %2.2f" % GBC_accuracy_score)

GBC accuracy is 0.63


In [46]:
print(classification_report(y_Sobel2_test, y_race_Sobel2_pred))

              precision    recall  f1-score   support

           0       0.65      0.94      0.77      1578
           1       0.62      0.11      0.19       119
           2       0.63      0.31      0.41       457
           3       0.57      0.44      0.50       445
           4       0.42      0.10      0.16       335

    accuracy                           0.63      2934
   macro avg       0.58      0.38      0.41      2934
weighted avg       0.61      0.63      0.58      2934



### 1.2.3 mask: Filter hsv -- Label age

In [47]:
X_Sobel2_train, X_Sobel2_test, y_Sobel2_train, y_Sobel2_test = train_test_split(Sobel2_2d, y3, test_size=0.3, random_state=42)

In [48]:
gbc.fit(X_Sobel2_train,y_Sobel2_train)

GradientBoostingClassifier(learning_rate=0.05, max_features=5, n_estimators=500,
                           random_state=100)

In [49]:
y_age_Sobel2_pred = gbc.predict(X_Sobel2_test)

In [50]:
print(confusion_matrix(y_Sobel2_test, y_age_Sobel2_pred))

[[425  12   0   0   8  21   3  15   1]
 [108  22   2   0   8  36   4  20   2]
 [ 30  13  11   1  25  45   2  19   0]
 [ 12   6   9   4  36  23   1  21   0]
 [ 14   8   3   5  95 137  11  55   1]
 [ 15   3   5   2  46 391  56  99   1]
 [  6   2   1   0  16  97  64 171   1]
 [ 18   2   2   1  18 102  35 376   2]
 [  2   0   1   0   0  20   2  95   8]]


In [51]:
GBC_accuracy_score = accuracy_score(y_Sobel2_test, y_age_Sobel2_pred)
print("GBC accuracy is %2.2f" % GBC_accuracy_score)

GBC accuracy is 0.48


In [52]:
print(classification_report(y_Sobel2_test, y_age_Sobel2_pred))

              precision    recall  f1-score   support

           1       0.67      0.88      0.76       485
           2       0.32      0.11      0.16       202
           3       0.32      0.08      0.12       146
           4       0.31      0.04      0.06       112
           5       0.38      0.29      0.33       329
           6       0.45      0.63      0.52       618
           7       0.36      0.18      0.24       358
           8       0.43      0.68      0.53       556
           9       0.50      0.06      0.11       128

    accuracy                           0.48      2934
   macro avg       0.42      0.33      0.32      2934
weighted avg       0.45      0.48      0.43      2934



### 1.3 mask: roberts 
### 1.3.1 mask: robert --- Label gender

In [20]:
Robert = list(robertsSobelEdges(imagesArray))
Robert1 = Robert[0]
Robert2= Robert[1]

In [48]:
Robert1.shape, Robert2.shape

((9778, 200, 200), (9778, 200, 200))

In [29]:
nsamples, nx, ny= Robert1.shape
Robert1_2d = Robert1.reshape((nsamples,nx*ny))

In [30]:
X_train, X_test, y_train, y_test = train_test_split(Robert1_2d, y1, test_size=0.3, random_state=42)

In [31]:
gbc.fit(X_train,y_train)

GradientBoostingClassifier(learning_rate=0.05, max_features=5, n_estimators=500,
                           random_state=100)

In [32]:
y_pred = gbc.predict(X_test)

In [33]:
print(confusion_matrix(y_test, y_pred))

[[ 892  393]
 [ 344 1305]]


In [34]:
GBC_accuracy_score = accuracy_score(y_test, y_pred)
print("GBC accuracy is %2.2f" % GBC_accuracy_score)

GBC accuracy is 0.75


In [35]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.72      0.69      0.71      1285
           1       0.77      0.79      0.78      1649

    accuracy                           0.75      2934
   macro avg       0.75      0.74      0.74      2934
weighted avg       0.75      0.75      0.75      2934



### 1.3.2 Mask Robert -- Label Race

In [36]:
X2_train, X2_test, y2_train, y2_test = train_test_split(Robert1_2d, y2, test_size=0.3, random_state=42)

In [37]:
gbc.fit(X2_train,y2_train)

GradientBoostingClassifier(learning_rate=0.05, max_features=5, n_estimators=500,
                           random_state=100)

In [38]:
y_race_pred = gbc.predict(X2_test)

In [39]:
print(confusion_matrix(y2_test, y_race_pred))

[[1485    1   50   34    8]
 [  91    3   10   14    1]
 [ 189    1  247   19    1]
 [ 234    0   20  186    5]
 [ 267    0   20   32   16]]


In [40]:
GBC_accuracy_score = accuracy_score(y2_test, y_race_pred)
print("GBC accuracy is %2.2f" % GBC_accuracy_score)

GBC accuracy is 0.66


In [41]:
print(classification_report(y2_test, y_race_pred))

              precision    recall  f1-score   support

           0       0.66      0.94      0.77      1578
           1       0.60      0.03      0.05       119
           2       0.71      0.54      0.61       457
           3       0.65      0.42      0.51       445
           4       0.52      0.05      0.09       335

    accuracy                           0.66      2934
   macro avg       0.63      0.39      0.41      2934
weighted avg       0.65      0.66      0.60      2934



### 1.3.3 Mask: Robert --- Label age

In [42]:
X_train, X_test, y_train, y_test = train_test_split(Robert1_2d, y3, test_size=0.3, random_state=42)

In [43]:
gbc.fit(X_train,y_train)

GradientBoostingClassifier(learning_rate=0.05, max_features=5, n_estimators=500,
                           random_state=100)

In [44]:
y_age_pred = gbc.predict(X_test)

In [45]:
print(confusion_matrix(y_test, y_age_pred))

[[434  21   1   0   1  13   2  13   0]
 [ 91  55   4   1  11  21   1  18   0]
 [ 20  36  12   1  30  36   1  10   0]
 [  7   8   7   1  41  21   6  20   1]
 [  6   7   3   3  87 162   9  51   1]
 [  4   2   1   0  32 458  47  74   0]
 [  3   1   1   0   9 132  59 153   0]
 [  3   2   2   0  16  71  51 395  16]
 [  0   0   0   0   1   5   2  89  31]]


In [46]:
GBC_accuracy_score = accuracy_score(y_test, y_age_pred)
print("GBC accuracy is %2.2f" % GBC_accuracy_score)

GBC accuracy is 0.52


In [47]:
print(classification_report(y_test, y_age_pred))

              precision    recall  f1-score   support

           1       0.76      0.89      0.82       485
           2       0.42      0.27      0.33       202
           3       0.39      0.08      0.14       146
           4       0.17      0.01      0.02       112
           5       0.38      0.26      0.31       329
           6       0.50      0.74      0.60       618
           7       0.33      0.16      0.22       358
           8       0.48      0.71      0.57       556
           9       0.63      0.24      0.35       128

    accuracy                           0.52      2934
   macro avg       0.45      0.38      0.37      2934
weighted avg       0.49      0.52      0.48      2934



### 1.4 mask: Sobel
### 1.4.1 mask: Sobel --- Label gender

In [50]:
nsamples, nx, ny= Robert2.shape
Robert2_2d = Robert2.reshape((nsamples,nx*ny))

In [51]:
X_train, X_test, y_train, y_test = train_test_split(Robert2_2d, y1, test_size=0.3, random_state=42)

In [52]:
gbc.fit(X_train,y_train)

GradientBoostingClassifier(learning_rate=0.05, max_features=5, n_estimators=500,
                           random_state=100)

In [53]:
y_pred = gbc.predict(X_test)

In [56]:
GBC_accuracy_score = accuracy_score(y_test, y_pred)
print("GBC accuracy is %2.2f" % GBC_accuracy_score)

GBC accuracy is 0.75


In [54]:
print(confusion_matrix(y_test, y_pred))

[[ 903  382]
 [ 359 1290]]


In [55]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.72      0.70      0.71      1285
           1       0.77      0.78      0.78      1649

    accuracy                           0.75      2934
   macro avg       0.74      0.74      0.74      2934
weighted avg       0.75      0.75      0.75      2934



### 1.4.2 Mask: Sobel --- Label race

In [57]:
X_train, X_test, y_train, y_test = train_test_split(Robert2_2d, y2, test_size=0.3, random_state=42)

In [58]:
gbc.fit(X_train,y_train)

GradientBoostingClassifier(learning_rate=0.05, max_features=5, n_estimators=500,
                           random_state=100)

In [59]:
y_pred = gbc.predict(X_test)

In [60]:
BC_accuracy_score = accuracy_score(y_test, y_pred)
print("GBC accuracy is %2.2f" % GBC_accuracy_score)

GBC accuracy is 0.75


In [61]:
print(confusion_matrix(y_test, y_pred))

[[1489    1   41   41    6]
 [  86    8   11   14    0]
 [ 182    1  252   21    1]
 [ 227    1   21  188    8]
 [ 275    1   21   24   14]]


In [62]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.66      0.94      0.78      1578
           1       0.67      0.07      0.12       119
           2       0.73      0.55      0.63       457
           3       0.65      0.42      0.51       445
           4       0.48      0.04      0.08       335

    accuracy                           0.66      2934
   macro avg       0.64      0.41      0.42      2934
weighted avg       0.65      0.66      0.61      2934



### 1.4.3 Mask: Sobel -- Label age

In [63]:
X_train, X_test, y_train, y_test = train_test_split(Robert2_2d, y3, test_size=0.3, random_state=42)

In [64]:
gbc.fit(X_train,y_train)

GradientBoostingClassifier(learning_rate=0.05, max_features=5, n_estimators=500,
                           random_state=100)

In [65]:
y_pred = gbc.predict(X_test)

In [66]:
BC_accuracy_score = accuracy_score(y_test, y_pred)
print("GBC accuracy is %2.2f" % GBC_accuracy_score)

GBC accuracy is 0.75


In [67]:
print(confusion_matrix(y_test, y_pred))

[[427  32   2   0   0  11   1  12   0]
 [ 98  50   3   2  13  20   1  15   0]
 [ 26  26  16   1  32  35   1   9   0]
 [  5  12   8   3  33  26   2  22   1]
 [  8   9   1   5  94 155  15  42   0]
 [  4   2   2   0  27 465  50  68   0]
 [  2   0   0   0   9 121  59 167   0]
 [  1   0   2   0  11  74  34 422  12]
 [  0   1   0   0   5   4   1  83  34]]


In [68]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           1       0.75      0.88      0.81       485
           2       0.38      0.25      0.30       202
           3       0.47      0.11      0.18       146
           4       0.27      0.03      0.05       112
           5       0.42      0.29      0.34       329
           6       0.51      0.75      0.61       618
           7       0.36      0.16      0.23       358
           8       0.50      0.76      0.60       556
           9       0.72      0.27      0.39       128

    accuracy                           0.54      2934
   macro avg       0.49      0.39      0.39      2934
weighted avg       0.51      0.54      0.49      2934



### 1.5 exposure
### 1.5.1 Mask: exposure --- Label gender

In [21]:
Exposure = exposure_(imagesArray)
Exposure1 = list(Exposure)

/Users/yinzi/opt/anaconda3/lib/python3.9/site-packages/skimage/exposure/exposure.py:181: UserWarning: This might be a color image. The histogram will be computed on the flattened image. You can instead apply this function to each color channel.
  hist, bin_centers = histogram(image, nbins)


In [24]:
Exposure1 = np.array(Exposure1)
Exposure1.shape

(9778, 200, 200, 3)

In [25]:
nsamples, nx, ny, n= Exposure1.shape
Exposure1_2d = Exposure1.reshape((nsamples,nx*ny*n))

In [26]:
X_train, X_test, y_train, y_test = train_test_split(Exposure1_2d, y1, test_size=0.3, random_state=42)

In [27]:
gbc.fit(X_train,y_train)

GradientBoostingClassifier(learning_rate=0.05, max_features=5, n_estimators=500,
                           random_state=100)

In [28]:
y5_gender_pred = gbc.predict(X_test)

In [30]:
GBC_accuracy_score = accuracy_score(y_test, y5_gender_pred)
print("GBC accuracy is %2.2f" % GBC_accuracy_score)

GBC accuracy is 0.78


In [31]:
print(confusion_matrix(y_test, y5_gender_pred))

[[ 961  324]
 [ 323 1326]]


In [33]:
print(classification_report(y_test, y5_gender_pred))

              precision    recall  f1-score   support

           0       0.75      0.75      0.75      1285
           1       0.80      0.80      0.80      1649

    accuracy                           0.78      2934
   macro avg       0.78      0.78      0.78      2934
weighted avg       0.78      0.78      0.78      2934



### 1.5.2 Mask: exposure --- Label race

In [44]:
X_train, X_test, y_train, y_test = train_test_split(Exposure1_2d, y2, test_size=0.3, random_state=42)

In [45]:
gbc.fit(X_train,y_train)

GradientBoostingClassifier(learning_rate=0.05, max_features=5, n_estimators=500,
                           random_state=100)

In [46]:
y5_race_pred = gbc.predict(X_test)

In [47]:
GBC_accuracy_score = accuracy_score(y_test, y5_race_pred)
print("GBC accuracy is %2.2f" % GBC_accuracy_score)

GBC accuracy is 0.70


In [48]:
print(confusion_matrix(y_test, y5_race_pred))

[[1481    0   40   52    5]
 [  63   22    8   19    7]
 [ 140    2  297   15    3]
 [ 179    3   25  225   13]
 [ 224    1   33   43   34]]


In [49]:
print(classification_report(y_test, y5_race_pred))

              precision    recall  f1-score   support

           0       0.71      0.94      0.81      1578
           1       0.79      0.18      0.30       119
           2       0.74      0.65      0.69       457
           3       0.64      0.51      0.56       445
           4       0.55      0.10      0.17       335

    accuracy                           0.70      2934
   macro avg       0.68      0.48      0.51      2934
weighted avg       0.69      0.70      0.66      2934



### 1.5.3 Mask: exposure --- Label Age

In [50]:
X_train, X_test, y_train, y_test = train_test_split(Exposure1_2d, y3, test_size=0.3, random_state=42)

In [51]:
gbc.fit(X_train,y_train)

GradientBoostingClassifier(learning_rate=0.05, max_features=5, n_estimators=500,
                           random_state=100)

In [52]:
y5_age_pred = gbc.predict(X_test)

In [53]:
GBC_accuracy_score = accuracy_score(y_test, y5_age_pred)
print("GBC accuracy is %2.2f" % GBC_accuracy_score)

GBC accuracy is 0.54


In [54]:
print(confusion_matrix(y_test, y5_age_pred))

[[444  16   1   1   2   7   3  10   1]
 [ 94  62   4   0  19  13   1   9   0]
 [ 17  27  14   4  38  35   4   7   0]
 [  9  14   7   2  34  30   5  10   1]
 [  4   8   5   5 110 151  13  32   1]
 [  4   6   1   0  28 445  62  72   0]
 [  1   3   0   0   7 123 103 121   0]
 [  2   1   2   2  16  76  58 393   6]
 [  1   0   0   0   5   7   4  93  18]]


In [55]:
print(classification_report(y_test, y5_age_pred))

              precision    recall  f1-score   support

           1       0.77      0.92      0.84       485
           2       0.45      0.31      0.37       202
           3       0.41      0.10      0.16       146
           4       0.14      0.02      0.03       112
           5       0.42      0.33      0.37       329
           6       0.50      0.72      0.59       618
           7       0.41      0.29      0.34       358
           8       0.53      0.71      0.60       556
           9       0.67      0.14      0.23       128

    accuracy                           0.54      2934
   macro avg       0.48      0.39      0.39      2934
weighted avg       0.52      0.54      0.50      2934



### 1.6 gammaLogaritmictCorrection
### 1.6.1 Mask Gamma1 -- Label gender

In [56]:
Gamma = list(gammaLogaritmictCorrection(imagesArray))
Gamma1 = Gamma[0]
Gamma2= Gamma[1]

In [57]:
Gamma1.shape, Gamma2.shape

((9778, 200, 200, 3), (9778, 200, 200, 3))

In [58]:
nsamples, nx, ny, n= Gamma1.shape
Gamma1_2d = Gamma1.reshape((nsamples,nx*ny*n))

In [59]:
X_train, X_test, y_train, y_test = train_test_split(Gamma1_2d, y1, test_size=0.3, random_state=42)

In [60]:
gbc.fit(X_train,y_train)

GradientBoostingClassifier(learning_rate=0.05, max_features=5, n_estimators=500,
                           random_state=100)

In [61]:
y6_gender_pred = gbc.predict(X_test)

In [62]:
GBC_accuracy_score = accuracy_score(y_test, y6_gender_pred)
print("GBC accuracy is %2.2f" % GBC_accuracy_score)

GBC accuracy is 0.77


In [63]:
print(confusion_matrix(y_test, y6_gender_pred))

[[ 928  357]
 [ 312 1337]]


In [64]:
print(classification_report(y_test, y6_gender_pred))

              precision    recall  f1-score   support

           0       0.75      0.72      0.74      1285
           1       0.79      0.81      0.80      1649

    accuracy                           0.77      2934
   macro avg       0.77      0.77      0.77      2934
weighted avg       0.77      0.77      0.77      2934



### 1.6.2 Mask Gamma1 -- Label race

In [65]:
X_train, X_test, y_train, y_test = train_test_split(Gamma1_2d, y2, test_size=0.3, random_state=42)

In [66]:
gbc.fit(X_train,y_train)

GradientBoostingClassifier(learning_rate=0.05, max_features=5, n_estimators=500,
                           random_state=100)

In [67]:
y6_race_pred = gbc.predict(X_test)

In [68]:
GBC_accuracy_score = accuracy_score(y_test, y6_race_pred)
print("GBC accuracy is %2.2f" % GBC_accuracy_score)

GBC accuracy is 0.71


In [69]:
print(confusion_matrix(y_test, y6_race_pred))

[[1449    0   47   58   24]
 [  51   36    1   24    7]
 [ 162    3  271   10   11]
 [ 148    6   17  259   15]
 [ 213    3   17   48   54]]


In [70]:
print(classification_report(y_test, y6_race_pred))

              precision    recall  f1-score   support

           0       0.72      0.92      0.80      1578
           1       0.75      0.30      0.43       119
           2       0.77      0.59      0.67       457
           3       0.65      0.58      0.61       445
           4       0.49      0.16      0.24       335

    accuracy                           0.71      2934
   macro avg       0.67      0.51      0.55      2934
weighted avg       0.69      0.71      0.68      2934



### 1.6.3 Gamma1 --- Label age

In [71]:
X_train, X_test, y_train, y_test = train_test_split(Gamma1_2d, y3, test_size=0.3, random_state=42)

In [72]:
gbc.fit(X_train,y_train)

GradientBoostingClassifier(learning_rate=0.05, max_features=5, n_estimators=500,
                           random_state=100)

In [73]:
y6_age_pred = gbc.predict(X_test)

In [74]:
GBC_accuracy_score = accuracy_score(y_test, y6_age_pred)
print("GBC accuracy is %2.2f" % GBC_accuracy_score)

GBC accuracy is 0.53


In [75]:
print(confusion_matrix(y_test, y6_race_pred))

[[  0   0   0   0   0   0   0   0   0   0]
 [201   2 200  64  18   0   0   0   0   0]
 [114   0  47  33   8   0   0   0   0   0]
 [ 99   2  20  18   7   0   0   0   0   0]
 [ 91   4   6   8   3   0   0   0   0   0]
 [237   3  17  46  26   0   0   0   0   0]
 [355  12  40 172  39   0   0   0   0   0]
 [303   9  10  30   6   0   0   0   0   0]
 [504  11  12  25   4   0   0   0   0   0]
 [119   5   1   3   0   0   0   0   0   0]]


In [76]:
print(classification_report(y_test, y6_age_pred))

              precision    recall  f1-score   support

           1       0.76      0.89      0.82       485
           2       0.44      0.28      0.34       202
           3       0.39      0.13      0.19       146
           4       0.44      0.07      0.12       112
           5       0.39      0.31      0.34       329
           6       0.50      0.70      0.59       618
           7       0.39      0.25      0.31       358
           8       0.50      0.71      0.59       556
           9       0.70      0.16      0.27       128

    accuracy                           0.53      2934
   macro avg       0.50      0.39      0.40      2934
weighted avg       0.52      0.53      0.50      2934



### 1.7 mask Gamma2
### 1.7.1 Mask: Gamma2 -- Label gender

In [77]:
nsamples, nx, ny, n= Gamma2.shape
Gamma2_2d = Gamma2.reshape((nsamples,nx*ny*n))

In [78]:
X_train, X_test, y_train, y_test = train_test_split(Gamma2_2d, y1, test_size=0.3, random_state=42)

In [79]:
gbc.fit(X_train,y_train)

GradientBoostingClassifier(learning_rate=0.05, max_features=5, n_estimators=500,
                           random_state=100)

In [80]:
y7_gender_pred = gbc.predict(X_test)

In [81]:
GBC_accuracy_score = accuracy_score(y_test, y7_gender_pred)
print("GBC accuracy is %2.2f" % GBC_accuracy_score)

GBC accuracy is 0.77


In [82]:
print(confusion_matrix(y_test, y7_gender_pred))

[[ 934  351]
 [ 319 1330]]


In [83]:
print(classification_report(y_test, y7_gender_pred))

              precision    recall  f1-score   support

           0       0.75      0.73      0.74      1285
           1       0.79      0.81      0.80      1649

    accuracy                           0.77      2934
   macro avg       0.77      0.77      0.77      2934
weighted avg       0.77      0.77      0.77      2934



### 1.7.2 Mask: Gamma2 -- Label race

In [84]:
X_train, X_test, y_train, y_test = train_test_split(Gamma2_2d, y2, test_size=0.3, random_state=42)

In [85]:
gbc.fit(X_train,y_train)

GradientBoostingClassifier(learning_rate=0.05, max_features=5, n_estimators=500,
                           random_state=100)

In [86]:
y7_race_pred = gbc.predict(X_test)

In [87]:
GBC_accuracy_score = accuracy_score(y_test, y7_race_pred)
print("GBC accuracy is %2.2f" % GBC_accuracy_score)

GBC accuracy is 0.70


In [88]:
print(confusion_matrix(y_test, y7_race_pred))

[[1451    0   48   58   21]
 [  50   34    3   26    6]
 [ 157    3  276   10   11]
 [ 152    4   18  256   15]
 [ 222    2   17   44   50]]


In [89]:
print(classification_report(y_test, y7_race_pred))

              precision    recall  f1-score   support

           0       0.71      0.92      0.80      1578
           1       0.79      0.29      0.42       119
           2       0.76      0.60      0.67       457
           3       0.65      0.58      0.61       445
           4       0.49      0.15      0.23       335

    accuracy                           0.70      2934
   macro avg       0.68      0.51      0.55      2934
weighted avg       0.69      0.70      0.67      2934



### 1.7.3 Gamma2 -- Label age

In [90]:
X_train, X_test, y_train, y_test = train_test_split(Gamma2_2d, y3, test_size=0.3, random_state=42)

In [91]:
gbc.fit(X_train,y_train)

GradientBoostingClassifier(learning_rate=0.05, max_features=5, n_estimators=500,
                           random_state=100)

In [92]:
y7_age_pred = gbc.predict(X_test)

In [93]:
GBC_accuracy_score = accuracy_score(y_test, y7_age_pred)
print("GBC accuracy is %2.2f" % GBC_accuracy_score)

GBC accuracy is 0.53


In [94]:
print(confusion_matrix(y_test, y7_age_pred))

[[433  24   1   0   5   7   1  14   0]
 [ 85  58   6   2  17  20   2  12   0]
 [ 24  23  16   3  27  32   2  19   0]
 [ 13   7   4   9  33  25   1  20   0]
 [  8   7   5   4  94 157  15  38   1]
 [  6   7   2   0  39 430  64  70   0]
 [  0   2   1   2  11 118 106 118   0]
 [  2   5   1   0  15  81  55 391   6]
 [  1   0   0   0   2   7   5  95  18]]


In [95]:
print(classification_report(y_test, y7_age_pred))

              precision    recall  f1-score   support

           1       0.76      0.89      0.82       485
           2       0.44      0.29      0.35       202
           3       0.44      0.11      0.18       146
           4       0.45      0.08      0.14       112
           5       0.39      0.29      0.33       329
           6       0.49      0.70      0.58       618
           7       0.42      0.30      0.35       358
           8       0.50      0.70      0.59       556
           9       0.72      0.14      0.24       128

    accuracy                           0.53      2934
   macro avg       0.51      0.39      0.39      2934
weighted avg       0.52      0.53      0.50      2934



### 1.8 histogramEqualizer
### 1.8.1 Mask Histogram Equalizer1 :   --- label gender

In [20]:
Histogram = list(histogramEqualizer(imagesArray))
Histogram1 = Histogram[0]
Histogram2 = Histogram[1]
Histogram3 = Histogram[2]

/Users/yinzi/opt/anaconda3/lib/python3.9/site-packages/skimage/exposure/exposure.py:181: UserWarning: This might be a color image. The histogram will be computed on the flattened image. You can instead apply this function to each color channel.
  hist, bin_centers = histogram(image, nbins)


In [21]:
nsamples, nx, ny, n= Histogram1.shape
Histogram1_2d = Histogram1.reshape((nsamples,nx*ny*n))

In [22]:
X_train, X_test, y_train, y_test = train_test_split(Histogram1_2d, y1, test_size=0.3, random_state=42)

In [23]:
gbc.fit(X_train,y_train)

GradientBoostingClassifier(learning_rate=0.05, max_features=5, n_estimators=500,
                           random_state=100)

In [24]:
y8_gender_pred = gbc.predict(X_test)

In [25]:
GBC_accuracy_score = accuracy_score(y_test, y8_gender_pred)
print("GBC accuracy is %2.2f" % GBC_accuracy_score)

GBC accuracy is 0.78


In [26]:
print(confusion_matrix(y_test, y8_gender_pred))

[[ 947  338]
 [ 302 1347]]


In [27]:
print(classification_report(y_test, y8_gender_pred))

              precision    recall  f1-score   support

           0       0.76      0.74      0.75      1285
           1       0.80      0.82      0.81      1649

    accuracy                           0.78      2934
   macro avg       0.78      0.78      0.78      2934
weighted avg       0.78      0.78      0.78      2934



### 1.8.2 Mask Histogram Equalizer 1 -- Label race

In [28]:
X_train, X_test, y_train, y_test = train_test_split(Histogram1_2d, y2, test_size=0.3, random_state=42)

In [29]:
gbc.fit(X_train,y_train)

GradientBoostingClassifier(learning_rate=0.05, max_features=5, n_estimators=500,
                           random_state=100)

In [30]:
y8_race_pred = gbc.predict(X_test)

In [31]:
GBC_accuracy_score = accuracy_score(y_test, y8_race_pred)
print("GBC accuracy is %2.2f" % GBC_accuracy_score)

GBC accuracy is 0.71


In [32]:
print(confusion_matrix(y_test, y8_race_pred))

[[1452    4   52   51   19]
 [  44   41    3   24    7]
 [ 138    3  288   15   13]
 [ 142    5   18  263   17]
 [ 217    3   25   37   53]]


In [33]:
print(classification_report(y_test, y8_race_pred))

              precision    recall  f1-score   support

           0       0.73      0.92      0.81      1578
           1       0.73      0.34      0.47       119
           2       0.75      0.63      0.68       457
           3       0.67      0.59      0.63       445
           4       0.49      0.16      0.24       335

    accuracy                           0.71      2934
   macro avg       0.67      0.53      0.57      2934
weighted avg       0.70      0.71      0.69      2934



### 1.8.3 Mask: Histogram Equalizer1 --- Label age

In [34]:
X_train, X_test, y_train, y_test = train_test_split(Histogram1_2d, y3, test_size=0.3, random_state=42)

In [35]:
gbc.fit(X_train,y_train)

GradientBoostingClassifier(learning_rate=0.05, max_features=5, n_estimators=500,
                           random_state=100)

In [36]:
y8_age_pred = gbc.predict(X_test)

In [37]:
GBC_accuracy_score = accuracy_score(y_test, y8_age_pred)
print("GBC accuracy is %2.2f" % GBC_accuracy_score)

GBC accuracy is 0.54


In [38]:
print(confusion_matrix(y_test, y8_age_pred))

[[431  27   0   1   5   6   1  12   2]
 [ 84  63  11   2  12  18   2  10   0]
 [ 17  25  20   2  39  31   4   8   0]
 [ 10   9  11   4  38  30   0   9   1]
 [  6   5   4   5 109 144  17  38   1]
 [  6   4   2   0  30 441  53  80   2]
 [  0   3   1   0   9 115 104 125   1]
 [  4   1   1   0  14  81  48 397  10]
 [  0   0   0   0   4   8   4  91  21]]


In [39]:
print(classification_report(y_test, y8_age_pred))

              precision    recall  f1-score   support

           1       0.77      0.89      0.83       485
           2       0.46      0.31      0.37       202
           3       0.40      0.14      0.20       146
           4       0.29      0.04      0.06       112
           5       0.42      0.33      0.37       329
           6       0.50      0.71      0.59       618
           7       0.45      0.29      0.35       358
           8       0.52      0.71      0.60       556
           9       0.55      0.16      0.25       128

    accuracy                           0.54      2934
   macro avg       0.48      0.40      0.40      2934
weighted avg       0.52      0.54      0.51      2934



### 1.9 Histogram Equalizer 2
### 1.9.1 Mask: Histogram Equalizer 2 --- Label: gender

In [40]:
nsamples, nx, ny, n= Histogram2.shape
Histogram2_2d = Histogram2.reshape((nsamples,nx*ny*n))

In [41]:
X_train, X_test, y_train, y_test = train_test_split(Histogram2_2d, y1, test_size=0.3, random_state=42)

In [42]:
gbc.fit(X_train,y_train)

GradientBoostingClassifier(learning_rate=0.05, max_features=5, n_estimators=500,
                           random_state=100)

In [43]:
y9_gender_pred = gbc.predict(X_test)

In [44]:
GBC_accuracy_score = accuracy_score(y_test, y9_gender_pred)
print("GBC accuracy is %2.2f" % GBC_accuracy_score)

GBC accuracy is 0.78


In [45]:
print(confusion_matrix(y_test, y9_gender_pred))

[[ 961  324]
 [ 323 1326]]


In [46]:
print(classification_report(y_test, y9_gender_pred))

              precision    recall  f1-score   support

           0       0.75      0.75      0.75      1285
           1       0.80      0.80      0.80      1649

    accuracy                           0.78      2934
   macro avg       0.78      0.78      0.78      2934
weighted avg       0.78      0.78      0.78      2934



### 1.9.2 Mask: Histogram Equalizer 2 --- Label: race 

In [47]:
X_train, X_test, y_train, y_test = train_test_split(Histogram2_2d, y2, test_size=0.3, random_state=42)

In [48]:
gbc.fit(X_train,y_train)

GradientBoostingClassifier(learning_rate=0.05, max_features=5, n_estimators=500,
                           random_state=100)

In [49]:
y9_race_pred = gbc.predict(X_test)

In [50]:
GBC_accuracy_score = accuracy_score(y_test, y9_race_pred)
print("GBC accuracy is %2.2f" % GBC_accuracy_score)

GBC accuracy is 0.70


In [51]:
print(confusion_matrix(y_test, y9_race_pred))

[[1481    0   40   52    5]
 [  63   22    8   19    7]
 [ 140    2  297   15    3]
 [ 179    3   25  225   13]
 [ 224    1   33   43   34]]


In [52]:
print(classification_report(y_test, y9_race_pred))

              precision    recall  f1-score   support

           0       0.71      0.94      0.81      1578
           1       0.79      0.18      0.30       119
           2       0.74      0.65      0.69       457
           3       0.64      0.51      0.56       445
           4       0.55      0.10      0.17       335

    accuracy                           0.70      2934
   macro avg       0.68      0.48      0.51      2934
weighted avg       0.69      0.70      0.66      2934



### 1.9.3 Mask: Histogram Equalizer 2 --- Label : age

In [53]:
X_train, X_test, y_train, y_test = train_test_split(Histogram2_2d, y3, test_size=0.3, random_state=42)

In [54]:
gbc.fit(X_train,y_train)

GradientBoostingClassifier(learning_rate=0.05, max_features=5, n_estimators=500,
                           random_state=100)

In [55]:
y9_age_pred = gbc.predict(X_test)

In [56]:
GBC_accuracy_score = accuracy_score(y_test, y9_age_pred)
print("GBC accuracy is %2.2f" % GBC_accuracy_score)

GBC accuracy is 0.54


In [57]:
print(confusion_matrix(y_test, y9_age_pred))

[[444  16   1   1   2   7   3  10   1]
 [ 94  62   4   0  19  13   1   9   0]
 [ 17  27  14   4  38  35   4   7   0]
 [  9  14   7   2  34  30   5  10   1]
 [  4   8   5   5 110 151  13  32   1]
 [  4   6   1   0  28 445  62  72   0]
 [  1   3   0   0   7 123 103 121   0]
 [  2   1   2   2  16  76  58 393   6]
 [  1   0   0   0   5   7   4  93  18]]


In [58]:
print(classification_report(y_test, y9_age_pred))

              precision    recall  f1-score   support

           1       0.77      0.92      0.84       485
           2       0.45      0.31      0.37       202
           3       0.41      0.10      0.16       146
           4       0.14      0.02      0.03       112
           5       0.42      0.33      0.37       329
           6       0.50      0.72      0.59       618
           7       0.41      0.29      0.34       358
           8       0.53      0.71      0.60       556
           9       0.67      0.14      0.23       128

    accuracy                           0.54      2934
   macro avg       0.48      0.39      0.39      2934
weighted avg       0.52      0.54      0.50      2934



### 1.10 Histogram Equalizer 3
### 1.10.1 Histogram Equalizer 3 --- label gender

In [59]:
nsamples, nx, ny, n= Histogram3.shape
Histogram3_2d = Histogram3.reshape((nsamples,nx*ny*n))

In [60]:
X_train, X_test, y_train, y_test = train_test_split(Histogram3_2d, y1, test_size=0.3, random_state=42)

In [61]:
gbc.fit(X_train,y_train)

GradientBoostingClassifier(learning_rate=0.05, max_features=5, n_estimators=500,
                           random_state=100)

In [62]:
y10_gender_pred = gbc.predict(X_test)

In [63]:
GBC_accuracy_score = accuracy_score(y_test, y10_gender_pred)
print("GBC accuracy is %2.2f" % GBC_accuracy_score)

GBC accuracy is 0.79


In [64]:
print(confusion_matrix(y_test, y10_gender_pred))

[[ 967  318]
 [ 287 1362]]


In [65]:
print(classification_report(y_test, y10_gender_pred))

              precision    recall  f1-score   support

           0       0.77      0.75      0.76      1285
           1       0.81      0.83      0.82      1649

    accuracy                           0.79      2934
   macro avg       0.79      0.79      0.79      2934
weighted avg       0.79      0.79      0.79      2934



### 1.10.2 Histogram Equalizer 3 --- Label race

In [66]:
X_train, X_test, y_train, y_test = train_test_split(Histogram3_2d, y2, test_size=0.3, random_state=42)

In [67]:
gbc.fit(X_train,y_train)

GradientBoostingClassifier(learning_rate=0.05, max_features=5, n_estimators=500,
                           random_state=100)

In [68]:
y10_race_pred = gbc.predict(X_test)

In [69]:
GBC_accuracy_score = accuracy_score(y_test, y10_race_pred)
print("GBC accuracy is %2.2f" % GBC_accuracy_score)

GBC accuracy is 0.71


In [70]:
print(confusion_matrix(y_test, y10_race_pred))

[[1486    1   38   41   12]
 [  49   30    5   27    8]
 [ 139    4  292   12   10]
 [ 160    1   26  244   14]
 [ 227    5   22   39   42]]


In [71]:
print(classification_report(y_test, y10_race_pred))

              precision    recall  f1-score   support

           0       0.72      0.94      0.82      1578
           1       0.73      0.25      0.37       119
           2       0.76      0.64      0.70       457
           3       0.67      0.55      0.60       445
           4       0.49      0.13      0.20       335

    accuracy                           0.71      2934
   macro avg       0.68      0.50      0.54      2934
weighted avg       0.69      0.71      0.68      2934



### 1.10.3 Histogram Equalizer 3 --- Label age

In [72]:
X_train, X_test, y_train, y_test = train_test_split(Histogram3_2d, y3, test_size=0.3, random_state=42)

In [73]:
gbc.fit(X_train,y_train)

GradientBoostingClassifier(learning_rate=0.05, max_features=5, n_estimators=500,
                           random_state=100)

In [74]:
y10_age_pred = gbc.predict(X_test)

In [75]:
GBC_accuracy_score = accuracy_score(y_test, y10_age_pred)
print("GBC accuracy is %2.2f" % GBC_accuracy_score)

GBC accuracy is 0.54


In [76]:
print(confusion_matrix(y_test, y10_age_pred))

[[439  16   1   1   0   9   0  19   0]
 [ 87  58  10   0  19  19   1   8   0]
 [ 21  25  16   3  41  28   4   8   0]
 [  7   8   4   4  48  24   4  13   0]
 [  6   9   5   7 107 147  10  38   0]
 [  5   7   1   0  35 434  63  73   0]
 [  0   4   1   0   9 126  92 126   0]
 [  4   1   0   1  13  93  28 408   8]
 [  0   1   0   0   2   3   3  98  21]]


In [77]:
print(classification_report(y_test, y10_age_pred))

              precision    recall  f1-score   support

           1       0.77      0.91      0.83       485
           2       0.45      0.29      0.35       202
           3       0.42      0.11      0.17       146
           4       0.25      0.04      0.06       112
           5       0.39      0.33      0.35       329
           6       0.49      0.70      0.58       618
           7       0.45      0.26      0.33       358
           8       0.52      0.73      0.61       556
           9       0.72      0.16      0.27       128

    accuracy                           0.54      2934
   macro avg       0.50      0.39      0.39      2934
weighted avg       0.52      0.54      0.50      2934

